In [740]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from statistics import mean, median, stdev
from math import sqrt
from modAL.models import ActiveLearner, CommitteeRegressor
from modAL.disagreement import max_std_sampling
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from math import sqrt

In [723]:
def generate_config():
    asm = ['', '--no-asm ']
    x8dct = ['', '--no-8x8dct ']
    cabac = ['', '--no-cabac ']
    deblock = ['', '--no-deblock ']
    pskip = ['', '--no-fast-pskip ']
    mbtree = ['', '--no-mbtree ']
    mixed_refs = ['', '--no-mixed-refs ']
    weightb = ['', '--no-weightb ']
    rc_lookahead = ['--rc-lookahead 20 ', '--rc-lookahead 40 ']
    rc_value = [20, 40]
    ref = ['--ref 1 ', '--ref 5 ', '--ref 9 ']
    ref_value = [1, 5, 9]
    eye_2 = np.eye(2)
    config_option = []
    all_possible_configs = []
    for element0 in asm:
        for element1 in x8dct:
            for element2 in cabac:
                for element3 in deblock:
                    for element4 in pskip:
                        for element5 in mbtree:
                            for element6 in mixed_refs:
                                for element7 in weightb:
                                    for element8 in rc_lookahead:
                                        for element9 in ref:
                                            _cmd = '../x264/x264 '
                                            _cmd = _cmd + element0
                                            _cmd = _cmd + element1
                                            _cmd = _cmd + element2
                                            _cmd = _cmd + element3
                                            _cmd = _cmd + element4
                                            _cmd = _cmd + element5
                                            _cmd = _cmd + element6
                                            _cmd = _cmd + element7
                                            _cmd = _cmd + element8
                                            _cmd = _cmd + element9
                                            config_option.append(_cmd)
                                            v0 = eye_2[asm.index(element0)]
                                            v1 = eye_2[x8dct.index(element1)]
                                            v2 = eye_2[cabac.index(element2)]
                                            v3 = eye_2[deblock.index(element3)]
                                            v4 = eye_2[pskip.index(element4)]
                                            v5 = eye_2[mbtree.index(element5)]
                                            v6 = eye_2[mixed_refs.index(element6)]
                                            v7 = eye_2[weightb.index(element7)]
                                            v8 = float(rc_value[rc_lookahead.index(element8)])
                                            v9 = float(ref_value[ref.index(element9)])
                                            all_possible_config = np.concatenate((v0, v1, v2, v3, v4, v5, v6, v7, v8, v9), axis=None)
                                            all_possible_configs.append(all_possible_config)
                                            # print(_cmd)
                                            config_option.append(_cmd)
    return config_option, all_possible_configs


def transfer_config(all_possible_configs):
    config_features = all_possible_configs
    config_features = np.asarray(config_features)
    scaler = MinMaxScaler()
    # scaler = StandardScaler()
    scaler.fit(config_features)
    config_features = scaler.transform(config_features)
    return config_features

In [724]:
lrzip_config, config_signal = generate_config()
all_input_signal = transfer_config(config_signal)
all_data = pd.read_csv("./x264.csv", index_col=0)
all_possible_configs_cur = np.asarray(config_signal)
config_features = np.asarray(all_input_signal)

In [244]:
res_al_all_x264 = []
for commit_time in range(0, len(training_num_set)):
    print(commit_time)
    real_result = np.asarray(all_data[all_data['commit_num'] == commit_time]['time'])
    res_al = []
    for seed_num in range(1,102,5):
        np.random.seed(seed_num)
        sampled_config_ids = list(np.random.randint(len(real_result), size=3))
        initial_idx = np.array_split(sampled_config_ids, 2)
        learner_list = [ActiveLearner(
                            estimator=XGBRegressor(),
                            X_training=config_features[idx], y_training=results[idx]
                    )
                    for idx in initial_idx]

        # initializing the Committee
        committee = CommitteeRegressor(
            learner_list=learner_list,
            query_strategy=max_std_sampling
        )

        model=XGBRegressor()
        n_queries = 7
        mean_re = []
        # median_re = []
        for idx in range(n_queries):
            X_train = config_features[sampled_config_ids]
            y_train = real_result[sampled_config_ids]
            X_test = config_features[~np.isin(np.arange(len(config_features)), selected_id)]
            y_test = real_result[~np.isin(np.arange(len(config_features)), selected_id)]
            model.fit(X_train, y_train)
            relative_error = []
            Y_predict = model.predict(X_test)
            relative_error = []
            for i in range(len(X_test)):
                RE=abs(y_test[i]-Y_predict[i])
                relative_error.append(RE*RE)    
            query_idx, query_instance = committee.query(config_features)
            sampled_config_ids += list(query_idx)
            committee.teach(config_features[query_idx], real_result[query_idx])
        res_al.append(model.score(X_test,y_test))
    res_al_all_x264.append(mean(res_al))
print(res_al_all_x264)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
[0.9382197448398678, 0.907246053012799, 0.936316248823584, 0.9195624577547068, 0.9349216727049834, 0.9385195940644492, 0.9381725679565144, 0.9343353852456401, 0.9097285089375811, 0.9098757292238491, 0.9433461506958775, 0.927077673900285, 0.9355475828367958, 0.9377826828059349, 0.9480460180598328, 0.9415706710659337, 0.9388856284783947, 0.9351730814012018, 0.9378332850161142, 0.9404346708213589, 0.9284752468374171, 0.9327108505938596, 0.9114448548218144, 0.9319450903991962, 0.937634480263184, 0.9377953404277783, 0.9416139094294961, 0.9356015657365754, 0.9207727582805731, 0.9404274629644063, 0.9344234075486417, 0.9318472328454955, 0.9057463539721142, 0.9404742866373562, 0.9327422299571142, 0.9374697494673259, 0.9426177477427921, 0.9217131134794573, 0.9214370061842845, 0.9374679887296744, 0.9311406922510183, 0.9299533629424348, 0.9039215510751537, 0.9296710

# LLVM

In [732]:
from itertools import chain, combinations
def powerset(list_name):
    s = list(list_name)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def generate_config():
    config_option = []
    all_possible_configs = []
    configs = ['--gvn ', '--instcombine ', '--inline ', '--jump-threading ', '--simplifycfg ', '--sccp ', '--ipsccp ', '--licm ',
               '--iv-users ']
    all_combination = powerset(configs)
    for x in all_combination:
        if len(x) > 0:
            cmd = ''
            for item in x:
                cmd += item
            config_option.append(cmd)
    for a1 in range(0, 2):
        for a2 in range(0, 2):
            for a3 in range(0, 2):
                for a4 in range(0, 2):
                    for a5 in range(0, 2):
                        for a6 in range(0, 2):
                            for a7 in range(0, 2):
                                for a8 in range(0, 2):
                                    for a9 in range(0, 2):
                                        all_possible_configs.append([a1, a2, a3, a4, a5, a6, a7, a8, a9])
    all_possible_configs.remove([0]*9)
    return config_option, all_possible_configs


def transfer_config(all_possible_configs):
    config_features = np.asarray(all_possible_configs)
    return config_features

In [733]:
lrzip_config, config_signal = generate_config()
all_input_signal = transfer_config(config_signal)
all_data = pd.read_csv("./llvm.csv", index_col=0)
# results = np.asarray(all_data[all_data['commit_num'] == 1]['time'])
all_possible_configs_cur = np.asarray(config_signal)
config_features = np.asarray(all_input_signal)

In [141]:
res_al_all_llvm = []
for commit_time in range(0, len(training_num_set)):
    print(commit_time)
    real_result = np.asarray(all_data[all_data['commit_num'] == commit_time]['time'])
    res_al = []
    for seed_num in range(1,102,5):
        np.random.seed(seed_num)
        sampled_config_ids = list(np.random.randint(len(real_result), size=6))
        initial_idx = np.array_split(sampled_config_ids, 2)
        learner_list = [ActiveLearner(
                            estimator=XGBRegressor(),
                            X_training=config_features[idx], y_training=results[idx]
                    )
                    for idx in initial_idx]

        # initializing the Committee
        committee = CommitteeRegressor(
            learner_list=learner_list,
            query_strategy=max_std_sampling
        )

        model=XGBRegressor()
        n_queries = 30
        mean_re = []
        # median_re = []
        for idx in range(n_queries):
            X_train = config_features[sampled_config_ids]
            y_train = real_result[sampled_config_ids]
            X_test = config_features[~np.isin(np.arange(len(config_features)), selected_id)]
            y_test = real_result[~np.isin(np.arange(len(config_features)), selected_id)]
            model.fit(X_train, y_train)
            relative_error = []
            Y_predict = model.predict(X_test)
            relative_error = []
            for i in range(len(X_test)):
                RE=abs(y_test[i]-Y_predict[i])
                relative_error.append(RE*RE)    
            query_idx, query_instance = committee.query(config_features)
            sampled_config_ids += list(query_idx)
            committee.teach(config_features[query_idx], real_result[query_idx])
        res_al.append(model.score(X_test,y_test))
    res_al_all_llvm.append(mean(res_al))
print(res_al_all_llvm)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
[0.25996301319907256, 0.2768517159621511, 0.2750009925980178, 0.2105784456804734, 0.2547015370236607, 0.2283519622636027, 0.25587972152735144, 0.2525619251343056, 0.2632192587747006, 0.22443057563638283, 0.23233232040017798, 0.26088218812196906, 0.2404982611631629, 0.2253305094938436, 0.24021851534811292, 0.22421995999184402, 0.24387926009836602, 0.2246829118058151, 0.2763447321753173, 0.26432089388723284, 0.3229983811017253, 0.21470488119890085, 0.23905665738483772, 0.24191804703263936, 0.29805307360405897, 0.23484179349643058, 0.28985444140015026, 0.2175876777231114, 0.2948466304198635, 0.22576514967845693, 0.2743782428600541, 0.2905570650925187, 0.204100486999171, 0.20094213067160696, 0.2388828868294803, 0.24709663981334962, 0.21920333126750513, 0.24371942452248738, 0.28896327872449634, 0.3067668545186326, 0.2280021597230488, 0.21

# LRZIP

In [850]:
def generate_config():
    config_option = []
    all_possible_configs = []
    for algorithm in ['-z', '-b', '-g', '-n', '-l']:
        for level in range(8, 10):
            for window in range(1, 100, 20):
                for nice in range(-20, 20, 8):
                    for processor in range(1, 5):
                        _cmd = 'sudo lrzip {} -L {} -w {} -N {} -p {}'.format(algorithm, level, window, nice, processor)
                        all_possible_configs.append([algorithm, level, window, nice, processor])
                        config_option.append(_cmd)
    return config_option, all_possible_configs


def transfer_config(all_possible_configs):
    fea_algo_feature = np.eye(5)
    fea_algo_list = ['-b', '-g', '-l', '-n', '-z']
    all_possible_configs_cur = all_possible_configs
    all_possible_configs_cur = np.asanyarray(all_possible_configs_cur)
    config_features = []
    for possible_config in all_possible_configs_cur:
        algo_feature = fea_algo_feature[fea_algo_list.index(possible_config[0])]
        config_features.append(np.concatenate([algo_feature, np.asarray(possible_config[1:], dtype=float)]))
    config_features = np.asarray(config_features)
    scaler = MinMaxScaler()
    # scaler = StandardScaler()
    scaler.fit(config_features)
    config_features = scaler.transform(config_features)
    return config_features

In [851]:
lrzip_config, config_signal = generate_config()
all_input_signal = transfer_config(config_signal)
all_data = pd.read_csv("./lrzip.csv", index_col=0)
results = np.asarray(all_data[all_data['commit_num'] == 1]['time'])
all_possible_configs_cur = np.asarray(config_signal)
config_features = np.asarray(all_input_signal)

In [852]:
res_al_all_lrzip = []
for commit_time in range(0, len(training_num_set)):
    print(commit_time)
    real_result = np.asarray(all_data[all_data['commit_num'] == commit_time]['time'])
    res_al = []
    for seed_num in range(1,102,5):
        np.random.seed(seed_num)
        sampled_config_ids = list(np.random.randint(len(real_result), size=6))
        initial_idx = np.array_split(sampled_config_ids, 3)
        learner_list = [ActiveLearner(
                            estimator=XGBRegressor(),
                            X_training=config_features[idx], y_training=results[idx]
                    )
                    for idx in initial_idx]

        # initializing the Committee
        committee = CommitteeRegressor(
            learner_list=learner_list,
            query_strategy=max_std_sampling
        )

        model=XGBRegressor()
        n_queries = 8
        mean_re = []
        # median_re = []
        for idx in range(n_queries):
            X_train = config_features[sampled_config_ids]
            y_train = real_result[sampled_config_ids]
            X_test = config_features[~np.isin(np.arange(len(config_features)), selected_id)]
            y_test = real_result[~np.isin(np.arange(len(config_features)), selected_id)]
            model.fit(X_train, y_train)
            relative_error = []
            Y_predict = model.predict(X_test)
            relative_error = []
            for i in range(len(X_test)):
                RE=abs(y_test[i]-Y_predict[i])
                relative_error.append(RE*RE)    
            query_idx, query_instance = committee.query(config_features)
            sampled_config_ids += list(query_idx)
            committee.teach(config_features[query_idx], real_result[query_idx])
        res_al.append(model.score(X_test,y_test))
    res_al_all_lrzip.append(mean(res_al))
print(res_al_all_lrzip)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
[0.8887021013055918, 0.799196542208493, 0.828288973980345, 0.8590695409455968, 0.8784447779839833, 0.8694425998541974, 0.9367863891555035, 0.9057380845788973, 0.8378398638316159, 0.8988652851949184, 0.8649465637697857, 0.8298373402432532, 0.8887223713566238, 0.8483655713750963, 0.8280002126835764, 0.8719761060843686, 0.8757726929030454, 0.899894047182921, 0.8728102513739835, 0.8800158800740233, 0.8715108907899277, 0.8138674576652033, 0.8075126724888768, 0.8618638949204611, 0.9415236946973264, 0.8006644114800798, 0.9257540999475073, 0.8323265635668076, 0.8359221862985836, 0.8631684114178338, 0.8050763230415788, 0.7963768333671787, 0.8258421711987612, 0.8962609519649809, 0.8929948656890858, 0.8525502569350153, 0.8413337532184111, 0.8739378614381849, 0.8740014889555318, 0.891701732000339, 0.9200187109707496, 0.89094664364

# sqlite

In [608]:
def generate_config():
    config_option = []
    all_possible_configs = []
    cache_size_list = [' -DSQLITE_DEFAULT_CACHE_SIZE=4000', ' -DSQLITE_DEFAULT_CACHE_SIZE=2000']
    auto_index_list = [' -DSQLITE_OMIT_AUTOMATIC_INDEX', '']
    page_size_list = [' -DSQLITE_DEFAULT_PAGE_SIZE=512', ' -DSQLITE_DEFAULT_PAGE_SIZE=1024', ' -DSQLITE_DEFAULT_PAGE_SIZE=2048']
    locking_mode_list = [' -DSQLITE_DEFAULT_LOCKING_MODE=0', ' -DSQLITE_DEFAULT_LOCKING_MODE=1']
    omit_feature_list = [' -DSQLITE_OMIT_AUTOMATIC_INDEX', ' -DSQLITE_OMIT_BTREECOUNT',
                                         ' -DSQLITE_OMIT_BETWEEN_OPTIMIZATION', ' -DSQLITE_OMIT_LIKE_OPTIMIZATION',
                                         ' -DSQLITE_OMIT_LOOKASIDE', ' -DSQLITE_OMIT_QUICKBALANCE', ' -DSQLITE_OMIT_OR_OPTIMIZATION',
                                         ' -DSQLITE_OMIT_SHARED_CACHE', ' -DSQLITE_OMIT_XFER_OPT']
    store_type_list = [' -DSQLITE_TEMP_STORE=0', ' -DSQLITE_TEMP_STORE=1', ' -DSQLITE_TEMP_STORE=2', ' -DSQLITE_TEMP_STORE=3']
    disable_feature_list = [' -DSQLITE_DISABLE_LFS', ' -DSQLITE_DISABLE_DIRSYNC']
    autovacuum_list = [' -DSQLITE_DEFAULT_AUTOVACUUM=0', ' -DSQLITE_DEFAULT_AUTOVACUUM=1']
    eye_2 = np.eye(2)
    eye_3 = np.eye(3)
    eye_4 = np.eye(4)
    eye_9 = np.eye(9)
    for cache_size in cache_size_list:
        for auto_index in auto_index_list:
            for page_size in page_size_list:
                for locking_mode in locking_mode_list:
                    for omit_feature in omit_feature_list:
                        for store_type in store_type_list:
                            for disable_feature in disable_feature_list:
                                for autovacuum in autovacuum_list:
                                    cmd = cache_size
                                    cmd += auto_index
                                    cmd += page_size
                                    cmd += locking_mode
                                    cmd += omit_feature
                                    cmd += store_type
                                    cmd += disable_feature
                                    cmd += autovacuum
                                    v0 = eye_2[cache_size_list.index(cache_size)]
                                    v1 = eye_2[auto_index_list.index(auto_index)]
                                    v2 = eye_3[page_size_list.index(page_size)]
                                    v3 = eye_2[locking_mode_list.index(locking_mode)]
                                    v4 = eye_9[omit_feature_list.index(omit_feature)]
                                    v5 = eye_4[store_type_list.index(store_type)]
                                    v6 = eye_2[disable_feature_list.index(disable_feature)]
                                    v7 = eye_2[autovacuum_list.index(autovacuum)]
                                    all_possible_config = np.concatenate((v0, v1, v2, v3, v4, v5, v6, v7), axis=None)
                                    all_possible_configs.append(all_possible_config)
                                    config_option.append(cmd)
    return config_option, all_possible_configs


def transfer_config(all_possible_configs):
    config_features = np.asarray(all_possible_configs)
    return config_features

In [609]:
lrzip_config, config_signal = generate_config()
all_input_signal = transfer_config(config_signal)
all_data = pd.read_csv("./sqlite.csv", index_col=0)
results = np.asarray(all_data[all_data['commit_num'] == 1]['time'])
all_possible_configs_cur = np.asarray(config_signal)
config_features = np.asarray(all_input_signal)

In [414]:
res_al_all_sqlite = []
for commit_time in range(0, len(training_num_set)):
    print(commit_time)
    real_result = np.asarray(all_data[all_data['commit_num'] == commit_time]['time'])
    res_al = []
    for seed_num in range(1,21):
        np.random.seed(seed_num)
        sampled_config_ids = list(np.random.randint(len(real_result), size=15))
        initial_idx = np.array_split(sampled_config_ids, 2)
        learner_list = [ActiveLearner(
                            estimator=XGBRegressor(),
                            X_training=config_features[idx], y_training=results[idx]
                    )
                    for idx in initial_idx]

        # initializing the Committee
        committee = CommitteeRegressor(
            learner_list=learner_list,
            query_strategy=max_std_sampling
        )

        model=XGBRegressor()
        n_queries = 42
        mean_re = []
        # median_re = []
        for idx in range(n_queries):
            X_train = config_features[sampled_config_ids]
            y_train = real_result[sampled_config_ids]
            X_test = config_features[~np.isin(np.arange(len(config_features)), selected_id)]
            y_test = real_result[~np.isin(np.arange(len(config_features)), selected_id)]
            model.fit(X_train, y_train)
            relative_error = []
            Y_predict = model.predict(X_test)
            relative_error = []
            for i in range(len(X_test)):
                RE=abs(y_test[i]-Y_predict[i])
                relative_error.append(RE*RE)    
            query_idx, query_instance = committee.query(config_features)
            sampled_config_ids += list(query_idx)
            committee.teach(config_features[query_idx], real_result[query_idx])
        res_al.append(model.score(X_test,y_test))
    res_al_all_sqlite.append(mean(res_al))
print(res_al_all_sqlite)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
[0.9206593486901228, 0.8638879193162279, 0.8769568799248403, 0.866349147483102, 0.9151616317551652, 0.9014299449681602, 0.9280529321935399, 0.899144850217508, 0.9099761236367538, 0.9152248210642954, 0.9122596112857756, 0.8721451252580785, 0.856024460470163, 0.9154533631642912, 0.8797699326925672, 0.8869232046826918, 0.9360648680757941, 0.8677402351084885, 0.8342813381499408, 0.8712803643572351, 0.8366687319478165, 0.9231964172593408, 0.8744842447005967, 0.9053573355231243, 0.8672697988831533, 0.8292726199664983, 0.8749968985804153, 0.7783607755651424, 0.7715601564972666, 0.7825533865868087, 0.9274488405917966, 0.7765864520122819, 0.8791515930502445, 0.8406261685123564, 0.908374809899222, 0.8205414761334637, 0.8998815011388299, 0.7527716371189755, 0.8111585662207265, 0.84417815397073, 0.9047753760182994, 0.7367404795989195, 0.8300105942175335, 0.

# LRZIP

In [1095]:
def generate_config():
    config_option = []
    all_possible_configs = []
    for algorithm in ['-z', '-b', '-g', '-n', '-l']:
        for level in range(8, 10):
            for window in range(1, 100, 20):
                for nice in range(-20, 20, 8):
                    for processor in range(1, 5):
                        _cmd = 'sudo lrzip {} -L {} -w {} -N {} -p {}'.format(algorithm, level, window, nice, processor)
                        all_possible_configs.append([algorithm, level, window, nice, processor])
                        config_option.append(_cmd)
    return config_option, all_possible_configs


def transfer_config(all_possible_configs):
    fea_algo_feature = np.eye(5)
    fea_algo_list = ['-b', '-g', '-l', '-n', '-z']
    all_possible_configs_cur = all_possible_configs
    all_possible_configs_cur = np.asanyarray(all_possible_configs_cur)
    config_features = []
    for possible_config in all_possible_configs_cur:
        algo_feature = fea_algo_feature[fea_algo_list.index(possible_config[0])]
        config_features.append(np.concatenate([algo_feature, np.asarray(possible_config[1:], dtype=float)]))
    config_features = np.asarray(config_features)
    scaler = MinMaxScaler()
    # scaler = StandardScaler()
    scaler.fit(config_features)
    config_features = scaler.transform(config_features)
    return config_features

In [1096]:
lrzip_config, config_signal = generate_config()
all_input_signal = transfer_config(config_signal)
all_data = pd.read_csv("./lrzip.csv", index_col=0)
results = np.asarray(all_data[all_data['commit_num'] == 1]['time'])
all_possible_configs_cur = np.asarray(config_signal)
config_features = np.asarray(all_input_signal)

In [1097]:
avg_lrzip_baseline=[0.799196542208493, 0.828288973980345, 0.8590695409455968, 0.8784447779839833, 
                    0.8694425998541974, 0.9367863891555035, 0.9057380845788973, 0.8378398638316159, 0.8988652851949184,
                    0.8649465637697857, 0.8298373402432532, 0.8887223713566238, 0.8483655713750963, 0.8280002126835764,
                    0.8719761060843686, 0.8757726929030454, 0.899894047182921, 0.8728102513739835, 0.8800158800740233,
                    0.8715108907899277, 0.8138674576652033, 0.8075126724888768, 0.8618638949204611, 0.9415236946973264,
                    0.8006644114800798, 0.9257540999475073, 0.8323265635668076, 0.8359221862985836, 0.8631684114178338,
                    0.8050763230415788, 0.7963768333671787, 0.8258421711987612, 0.8962609519649809, 0.8929948656890858,
                    0.8525502569350153, 0.8413337532184111, 0.8739378614381849, 0.8740014889555318, 0.891701732000339,
                    0.9200187109707496, 0.8909466436495089, 0.856679548679232, 0.8350823459411576, 0.8592386909881382,
                    0.8770350572358876, 0.8881577078066007, 0.9034584657520687, 0.8448523601606462, 0.8742618774384513,
                    0.9212717717710572, 0.8505719143571474, 0.8832005243912598, 0.779983929550607, 0.8660145124955007,
                    0.8634287588943821, 0.831866432882176, 0.8705433172690017, 0.820908564207273, 0.8707926727124687]

In [1114]:
file = open("lrzip_kmean_weighted_limited.0", 'r')
lines = file.readlines()
training_num = []
init_ids = []
for line in lines:
    if line.startswith('query idx'):
        idx_num = line[10:].replace('[', '')
        idx_num = idx_num.replace(']', '')
        idx_num = idx_num.replace('\n', '')
        training_num.append(int(idx_num))
    if line.startswith('sampled_config_ids'):
        all_elements = line.split(' ')[1:]
        nums = []
        for num in all_elements:
            num = num.replace('[', '')
            num = num.replace(']', '')
            num = num.replace('\n', '')
            num = num.replace(',', '')
            num = int(num)
            nums.append(num)
        nums = np.array(nums, dtype='int')
        init_ids.append(nums)
print(len(training_num))
print(len(init_ids))

1080
60


In [1117]:
index = training_num_set
r2_list=[]
r2_list_1=[]
l_num=[]
cut_num = 18
for commit_time in range(1, len(training_num_set)):
# for commit_time in range(24,27):  
    real_result = np.asarray(all_data[all_data['commit_num'] == commit_time]['time'])
    past_real_result = np.asarray(all_data[all_data['commit_num'] == commit_time-1]['time'])
    print('commit_time:' + str(commit_time))
    base_x_train = config_features[init_ids[commit_time]]
    base_y_train = past_real_result[init_ids[commit_time]]
    sub_selected_id = index[commit_time][9:18]
    # sub_selected_id = sub_selected_id[0:cut_num]
    # sub_selected_id = index[commit_time]
    for l in range(len(sub_selected_id)):
        extra_x_train = config_features[sub_selected_id[0:l]]
        extra_y_train = real_result[sub_selected_id[0:l]]
        X_train = np.concatenate((base_x_train,extra_x_train))
        y_train = np.concatenate((base_y_train,extra_y_train))
        X_test = config_features
        y_test = real_result
        model=XGBRegressor()
        model.fit(X_train, y_train)
        Y_predict = model.predict(X_test)
        # print('res_al: ' + str(res_al_all[commit_time]))
        # print('score: ' + str(model.score(X_test,y_test)))
        if model.score(X_test,y_test) > avg_lrzip_baseline[commit_time-1]:
            l_num.append(len(X_train)-len(init_ids[commit_time]))
            print('l: ' + str(l_num[-1]))
            print(model.score(X_test,y_test))
            print(lrzip_baseline[commit_time-1])
            break
        #print(model.score(X_test,y_test))
        #r2_list.append(model.score(X_test,y_test))
        #print(res_al_all_sqlite[commit_time])
        #r2_list_1.append(res_al_all_sqlite[commit_time])
#if commit_time > 0 and len(sub_selected_id) > 0:
#    selected_id = sub_selected_id[0:cut_num]

commit_time:1
l: 3
0.9911167847583812
0.591434530296503
commit_time:2
l: 3
0.8984152710162113
0.9977099086663398
commit_time:3
l: 4
0.8639293283773917
0.6076188477937052
commit_time:4
l: 1
0.9915296887834602
0.6184013003267179
commit_time:5
l: 3
0.870833183698312
0.9930428526991342
commit_time:6
commit_time:7
commit_time:8
l: 3
0.985422370989915
0.9937476029141397
commit_time:9
commit_time:10
l: 3
0.9845390717725812
0.9950585292814412
commit_time:11
l: 3
0.8645529870465867
0.9950890906636572
commit_time:12
commit_time:13
l: 3
0.852937307971556
0.6102192232847017
commit_time:14
l: 2
0.9814235530237165
0.6129909571328755
commit_time:15
commit_time:16
commit_time:17
commit_time:18
commit_time:19
commit_time:20
commit_time:21
l: 3
0.8574915969117052
0.9982044459074999
commit_time:22
l: 3
0.9802631851020268
0.6157756720730068
commit_time:23
l: 8
0.8626451644233506
0.9927023543461324
commit_time:24
commit_time:25
commit_time:26
commit_time:27
l: 3
0.8593867450198204
0.9980967683582626
commit

In [1119]:
np.percentile(l_num,95)

5.399999999999999

# LLVM

In [1045]:
from itertools import chain, combinations
def powerset(list_name):
    s = list(list_name)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def generate_config():
    config_option = []
    all_possible_configs = []
    configs = ['--gvn ', '--instcombine ', '--inline ', '--jump-threading ', '--simplifycfg ', '--sccp ', '--ipsccp ', '--licm ',
               '--iv-users ']
    all_combination = powerset(configs)
    for x in all_combination:
        if len(x) > 0:
            cmd = ''
            for item in x:
                cmd += item
            config_option.append(cmd)
    for a1 in range(0, 2):
        for a2 in range(0, 2):
            for a3 in range(0, 2):
                for a4 in range(0, 2):
                    for a5 in range(0, 2):
                        for a6 in range(0, 2):
                            for a7 in range(0, 2):
                                for a8 in range(0, 2):
                                    for a9 in range(0, 2):
                                        all_possible_configs.append([a1, a2, a3, a4, a5, a6, a7, a8, a9])
    all_possible_configs.remove([0]*9)
    return config_option, all_possible_configs


def transfer_config(all_possible_configs):
    config_features = np.asarray(all_possible_configs)
    return config_features

In [1046]:
lrzip_config, config_signal = generate_config()
all_input_signal = transfer_config(config_signal)
all_data = pd.read_csv("./llvm.csv", index_col=0)
# results = np.asarray(all_data[all_data['commit_num'] == 1]['time'])
all_possible_configs_cur = np.asarray(config_signal)
config_features = np.asarray(all_input_signal)

In [1069]:
file = open("./llvm_kmean_weighted_limited", 'r')
lines = file.readlines()
training_num = []
init_ids = []
for line in lines:
    if line.startswith('query idx'):
        idx_num = line[10:].replace('[', '')
        idx_num = idx_num.replace(']', '')
        idx_num = idx_num.replace('\n', '')
        training_num.append(int(idx_num))
    if line.startswith('sampled_config_ids'):
        all_elements = line.split(' ')[1:]
        nums = []
        for num in all_elements:
            num = num.replace('[', '')
            num = num.replace(']', '')
            num = num.replace('\n', '')
            num = num.replace(',', '')
            num = int(num)
            nums.append(num)
        nums = np.array(nums, dtype='int')
        init_ids.append(nums)
print(len(training_num))
print(len(init_ids))

3410
55


In [1070]:
training_num_set = np.array_split(training_num, 55)
training_num_set = np.array(training_num_set, dtype='int')

In [1226]:
avg_llvm_baseline = [0.2768517159621511, 0.2750009925980178, 0.2105784456804734, 
                     0.2547015370236607, 0.2283519622636027, 0.25587972152735144, 0.2525619251343056, 
                     0.2632192587747006, 0.22443057563638283, 0.23233232040017798, 0.26088218812196906, 
                     0.2404982611631629, 0.2253305094938436, 0.24021851534811292, 0.22421995999184402, 
                     0.24387926009836602, 0.2246829118058151, 0.2763447321753173, 0.26432089388723284, 
                     0.3229983811017253, 0.21470488119890085, 0.23905665738483772, 0.24191804703263936, 
                     0.29805307360405897, 0.23484179349643058, 0.28985444140015026, 0.2175876777231114, 
                     0.2948466304198635, 0.22576514967845693, 0.2743782428600541, 0.2905570650925187, 
                     0.204100486999171, 0.20094213067160696, 0.2388828868294803, 0.24709663981334962, 
                     0.21920333126750513, 0.24371942452248738, 0.28896327872449634, 0.3067668545186326, 
                     0.2280021597230488, 0.21850541108673902, 0.24260431342671646, 0.2519560895682281, 
                     0.24601348338235374, 0.2907778366248824, 0.27461244626926445, 0.2967584934994886, 
                     0.25012136523684203, 0.2808456181620018, 0.2739352595051297, 0.2995909476308452, 
                     0.2577661007327311, 0.2526515556898081, 0.2678748866594307]

In [1073]:
index = training_num_set
r2_list=[]
r2_list_1=[]
l_num=[]
cut_num = 40
for commit_time in range(1, len(training_num_set)):
# for commit_time in range(24,27):  
    real_result = np.asarray(all_data[all_data['commit_num'] == commit_time]['time'])
    past_real_result = np.asarray(all_data[all_data['commit_num'] == commit_time-1]['time'])
    print('commit_time:' + str(commit_time))
    base_x_train = config_features[init_ids[commit_time]]
    base_y_train = past_real_result[init_ids[commit_time]]
    sub_selected_id = index[commit_time][31:62]
    # sub_selected_id = sub_selected_id[0:cut_num]
    # sub_selected_id = index[commit_time]
    for l in range(len(sub_selected_id)):
        extra_x_train = config_features[sub_selected_id[0:l]]
        extra_y_train = real_result[sub_selected_id[0:l]]
        X_train = np.concatenate((base_x_train,extra_x_train))
        y_train = np.concatenate((base_y_train,extra_y_train))
        X_test = config_features
        y_test = real_result
        model=XGBRegressor()
        model.fit(X_train, y_train)
        Y_predict = model.predict(X_test)
        # print('res_al: ' + str(res_al_all[commit_time]))
        # print('score: ' + str(model.score(X_test,y_test)))
        if model.score(X_test,y_test) > avg_llvm_baseline[commit_time-1]:
            l_num.append(len(X_train)-len(init_ids[commit_time]))
            print('l: ' + str(l_num[-1]))
            print(model.score(X_test,y_test))
            print(avg_llvm_baseline[commit_time-1])
            break
        #print(model.score(X_test,y_test))
        #r2_list.append(model.score(X_test,y_test))
        #print(res_al_all_sqlite[commit_time])
        #r2_list_1.append(res_al_all_sqlite[commit_time])
#if commit_time > 0 and len(sub_selected_id) > 0:
#    selected_id = sub_selected_id[0:cut_num]

commit_time:1
l: 0
0.4419778519160962
0.2768517159621511
commit_time:2
l: 0
0.43693655154500555
0.2750009925980178
commit_time:3
l: 0
0.3707720719289014
0.2105784456804734
commit_time:4
l: 0
0.43529602778049037
0.2547015370236607
commit_time:5
l: 0
0.4180239500758287
0.2283519622636027
commit_time:6
l: 0
0.3404051265878769
0.25587972152735144
commit_time:7
l: 7
0.25937050208130175
0.2525619251343056
commit_time:8
l: 2
0.32606211516186534
0.2632192587747006
commit_time:9
l: 0
0.3066924332033838
0.22443057563638283
commit_time:10
l: 0
0.4444919192325437
0.23233232040017798
commit_time:11
l: 0
0.4177055336862838
0.26088218812196906
commit_time:12
l: 0
0.29163430746757346
0.2404982611631629
commit_time:13
l: 0
0.32000950676051454
0.2253305094938436
commit_time:14
l: 0
0.40173240024250867
0.24021851534811292
commit_time:15
l: 0
0.450287170897648
0.22421995999184402
commit_time:16
l: 0
0.3565701262780412
0.24387926009836602
commit_time:17
l: 0
0.48836992272994983
0.2246829118058151
commit_ti

In [1075]:
np.percentile(l_num,95)

7.699999999999989

# x264

In [1180]:
def generate_config():
    asm = ['', '--no-asm ']
    x8dct = ['', '--no-8x8dct ']
    cabac = ['', '--no-cabac ']
    deblock = ['', '--no-deblock ']
    pskip = ['', '--no-fast-pskip ']
    mbtree = ['', '--no-mbtree ']
    mixed_refs = ['', '--no-mixed-refs ']
    weightb = ['', '--no-weightb ']
    rc_lookahead = ['--rc-lookahead 20 ', '--rc-lookahead 40 ']
    rc_value = [20, 40]
    ref = ['--ref 1 ', '--ref 5 ', '--ref 9 ']
    ref_value = [1, 5, 9]
    eye_2 = np.eye(2)
    config_option = []
    all_possible_configs = []
    for element0 in asm:
        for element1 in x8dct:
            for element2 in cabac:
                for element3 in deblock:
                    for element4 in pskip:
                        for element5 in mbtree:
                            for element6 in mixed_refs:
                                for element7 in weightb:
                                    for element8 in rc_lookahead:
                                        for element9 in ref:
                                            _cmd = '../x264/x264 '
                                            _cmd = _cmd + element0
                                            _cmd = _cmd + element1
                                            _cmd = _cmd + element2
                                            _cmd = _cmd + element3
                                            _cmd = _cmd + element4
                                            _cmd = _cmd + element5
                                            _cmd = _cmd + element6
                                            _cmd = _cmd + element7
                                            _cmd = _cmd + element8
                                            _cmd = _cmd + element9
                                            config_option.append(_cmd)
                                            v0 = eye_2[asm.index(element0)]
                                            v1 = eye_2[x8dct.index(element1)]
                                            v2 = eye_2[cabac.index(element2)]
                                            v3 = eye_2[deblock.index(element3)]
                                            v4 = eye_2[pskip.index(element4)]
                                            v5 = eye_2[mbtree.index(element5)]
                                            v6 = eye_2[mixed_refs.index(element6)]
                                            v7 = eye_2[weightb.index(element7)]
                                            v8 = float(rc_value[rc_lookahead.index(element8)])
                                            v9 = float(ref_value[ref.index(element9)])
                                            all_possible_config = np.concatenate((v0, v1, v2, v3, v4, v5, v6, v7, v8, v9), axis=None)
                                            all_possible_configs.append(all_possible_config)
                                            # print(_cmd)
                                            config_option.append(_cmd)
    return config_option, all_possible_configs


def transfer_config(all_possible_configs):
    config_features = all_possible_configs
    config_features = np.asarray(config_features)
    scaler = MinMaxScaler()
    # scaler = StandardScaler()
    scaler.fit(config_features)
    config_features = scaler.transform(config_features)
    return config_features

In [1181]:
lrzip_config, config_signal = generate_config()
all_input_signal = transfer_config(config_signal)
all_data = pd.read_csv("./x264.csv", index_col=0)
all_possible_configs_cur = np.asarray(config_signal)
config_features = np.asarray(all_input_signal)

In [1182]:
file = open("x264_kmean_weighted_limited.0", 'r')
lines = file.readlines()
training_num = []
init_ids = []
for line in lines:
    if line.startswith('query idx'):
        idx_num = line[10:].replace('[', '')
        idx_num = idx_num.replace(']', '')
        idx_num = idx_num.replace('\n', '')
        training_num.append(int(idx_num))
    if line.startswith('sampled_config_ids'):
        all_elements = line.split(' ')[1:]
        nums = []
        for num in all_elements:
            num = num.replace('[', '')
            num = num.replace(']', '')
            num = num.replace('\n', '')
            num = num.replace(',', '')
            num = int(num)
            nums.append(num)
        nums = np.array(nums, dtype='int')
        init_ids.append(nums)
print(len(training_num))
print(len(init_ids))

576
48


In [1130]:
training_num_set = np.array_split(training_num, 48)
training_num_set = np.array(training_num_set, dtype='int')

In [1132]:
avg_x264_baseline=[0.9382197448398678, 0.907246053012799, 0.936316248823584, 0.9195624577547068, 0.9349216727049834,
                   0.9385195940644492, 0.9381725679565144, 0.9343353852456401, 0.9097285089375811, 0.9098757292238491,
                   0.9433461506958775, 0.927077673900285, 0.9355475828367958, 0.9377826828059349, 0.9480460180598328,
                   0.9415706710659337, 0.9388856284783947, 0.9351730814012018, 0.9378332850161142, 0.9404346708213589,
                   0.9284752468374171, 0.9327108505938596, 0.9114448548218144, 0.9319450903991962, 0.937634480263184,
                   0.9377953404277783, 0.9416139094294961, 0.9356015657365754, 0.9207727582805731, 0.9404274629644063,
                   0.9344234075486417, 0.9318472328454955, 0.9057463539721142, 0.9404742866373562, 0.9327422299571142,
                   0.9374697494673259, 0.9426177477427921, 0.9217131134794573, 0.9214370061842845, 0.9374679887296744,
                   0.9311406922510183, 0.9299533629424348, 0.9039215510751537, 0.929671054151725, 0.938287318205055, 
                   0.9364113060953411, 0.9512959037303043, 0.935795766020886]

In [1133]:
index = training_num_set
r2_list=[]
r2_list_1=[]
l_num=[]
cut_num = 15
for commit_time in range(1, len(training_num_set)):
# for commit_time in range(24,27):  
    real_result = np.asarray(all_data[all_data['commit_num'] == commit_time]['time'])
    past_real_result = np.asarray(all_data[all_data['commit_num'] == commit_time-1]['time'])
    print('commit_time:' + str(commit_time))
    base_x_train = config_features[init_ids[commit_time]]
    base_y_train = past_real_result[init_ids[commit_time]]
    sub_selected_id = index[commit_time][6:12]
    # sub_selected_id = sub_selected_id[0:cut_num]
    # sub_selected_id = index[commit_time]
    for l in range(len(sub_selected_id)):
        extra_x_train = config_features[sub_selected_id[0:l]]
        extra_y_train = real_result[sub_selected_id[0:l]]
        X_train = np.concatenate((base_x_train,extra_x_train))
        y_train = np.concatenate((base_y_train,extra_y_train))
        X_test = config_features
        y_test = real_result
        model=XGBRegressor()
        model.fit(X_train, y_train)
        Y_predict = model.predict(X_test)
        # print('res_al: ' + str(res_al_all[commit_time]))
        # print('score: ' + str(model.score(X_test,y_test)))
        if model.score(X_test,y_test) > avg_x264_baseline[commit_time-1]:
            l_num.append(len(X_train)-len(init_ids[commit_time]))
            print('l: ' + str(l_num[-1]))
            print(model.score(X_test,y_test))
            print(avg_x264_baseline[commit_time-1])
            break
        #print(model.score(X_test,y_test))
        #r2_list.append(model.score(X_test,y_test))
        #print(res_al_all_sqlite[commit_time])
        #r2_list_1.append(res_al_all_sqlite[commit_time])
#if commit_time > 0 and len(sub_selected_id) > 0:
#    selected_id = sub_selected_id[0:cut_num]

commit_time:1
l: 0
0.9758011892585394
0.9382197448398678
commit_time:2
l: 0
0.9478013173145828
0.907246053012799
commit_time:3
l: 0
0.9817615555846597
0.936316248823584
commit_time:4
l: 0
0.9792896317994211
0.9195624577547068
commit_time:5
l: 0
0.9765958518290495
0.9349216727049834
commit_time:6
l: 0
0.9403506958644623
0.9385195940644492
commit_time:7
l: 0
0.9891201938292487
0.9381725679565144
commit_time:8
l: 3
0.9778262548679759
0.9343353852456401
commit_time:9
l: 0
0.9368454114547105
0.9097285089375811
commit_time:10
l: 0
0.9384744684052321
0.9098757292238491
commit_time:11
l: 0
0.9782452856898668
0.9433461506958775
commit_time:12
l: 0
0.9838662630953344
0.927077673900285
commit_time:13
l: 0
0.9889444697901755
0.9355475828367958
commit_time:14
l: 0
0.949036757138974
0.9377826828059349
commit_time:15
l: 0
0.9806691662993274
0.9480460180598328
commit_time:16
l: 1
0.9439469998305863
0.9415706710659337
commit_time:17
l: 3
0.9884580282770401
0.9388856284783947
commit_time:18
l: 0
0.93533

In [1135]:
np.percentile(l_num,95)

3.0

In [1136]:
len(model.feature_importances_)

18

# sqlite

In [1155]:
def generate_config():
    config_option = []
    all_possible_configs = []
    cache_size_list = [' -DSQLITE_DEFAULT_CACHE_SIZE=4000', ' -DSQLITE_DEFAULT_CACHE_SIZE=2000']
    auto_index_list = [' -DSQLITE_OMIT_AUTOMATIC_INDEX', '']
    page_size_list = [' -DSQLITE_DEFAULT_PAGE_SIZE=512', ' -DSQLITE_DEFAULT_PAGE_SIZE=1024', ' -DSQLITE_DEFAULT_PAGE_SIZE=2048']
    locking_mode_list = [' -DSQLITE_DEFAULT_LOCKING_MODE=0', ' -DSQLITE_DEFAULT_LOCKING_MODE=1']
    omit_feature_list = [' -DSQLITE_OMIT_AUTOMATIC_INDEX', ' -DSQLITE_OMIT_BTREECOUNT',
                                         ' -DSQLITE_OMIT_BETWEEN_OPTIMIZATION', ' -DSQLITE_OMIT_LIKE_OPTIMIZATION',
                                         ' -DSQLITE_OMIT_LOOKASIDE', ' -DSQLITE_OMIT_QUICKBALANCE', ' -DSQLITE_OMIT_OR_OPTIMIZATION',
                                         ' -DSQLITE_OMIT_SHARED_CACHE', ' -DSQLITE_OMIT_XFER_OPT']
    store_type_list = [' -DSQLITE_TEMP_STORE=0', ' -DSQLITE_TEMP_STORE=1', ' -DSQLITE_TEMP_STORE=2', ' -DSQLITE_TEMP_STORE=3']
    disable_feature_list = [' -DSQLITE_DISABLE_LFS', ' -DSQLITE_DISABLE_DIRSYNC']
    autovacuum_list = [' -DSQLITE_DEFAULT_AUTOVACUUM=0', ' -DSQLITE_DEFAULT_AUTOVACUUM=1']
    eye_2 = np.eye(2)
    eye_3 = np.eye(3)
    eye_4 = np.eye(4)
    eye_9 = np.eye(9)
    for cache_size in cache_size_list:
        for auto_index in auto_index_list:
            for page_size in page_size_list:
                for locking_mode in locking_mode_list:
                    for omit_feature in omit_feature_list:
                        for store_type in store_type_list:
                            for disable_feature in disable_feature_list:
                                for autovacuum in autovacuum_list:
                                    cmd = cache_size
                                    cmd += auto_index
                                    cmd += page_size
                                    cmd += locking_mode
                                    cmd += omit_feature
                                    cmd += store_type
                                    cmd += disable_feature
                                    cmd += autovacuum
                                    v0 = eye_2[cache_size_list.index(cache_size)]
                                    v1 = eye_2[auto_index_list.index(auto_index)]
                                    v2 = eye_3[page_size_list.index(page_size)]
                                    v3 = eye_2[locking_mode_list.index(locking_mode)]
                                    v4 = eye_9[omit_feature_list.index(omit_feature)]
                                    v5 = eye_4[store_type_list.index(store_type)]
                                    v6 = eye_2[disable_feature_list.index(disable_feature)]
                                    v7 = eye_2[autovacuum_list.index(autovacuum)]
                                    all_possible_config = np.concatenate((v0, v1, v2, v3, v4, v5, v6, v7), axis=None)
                                    all_possible_configs.append(all_possible_config)
                                    config_option.append(cmd)
    return config_option, all_possible_configs


def transfer_config(all_possible_configs):
    config_features = np.asarray(all_possible_configs)
    return config_features

In [1156]:
lrzip_config, config_signal = generate_config()
all_input_signal = transfer_config(config_signal)
all_data = pd.read_csv("./sqlite.csv", index_col=0)
results = np.asarray(all_data[all_data['commit_num'] == 1]['time'])
all_possible_configs_cur = np.asarray(config_signal)
config_features = np.asarray(all_input_signal)

In [1171]:
file = open("sqlite_weighted_limited_version.0", 'r')
lines = file.readlines()
training_num = []
init_ids = []
for line in lines:
    if line.startswith('query idx'):
        idx_num = line[10:].replace('[', '')
        idx_num = idx_num.replace(']', '')
        idx_num = idx_num.replace('\n', '')
        training_num.append(int(idx_num))
    if line.startswith('sampled_config_ids'):
        all_elements = line.split(' ')[1:]
        nums = []
        for num in all_elements:
            num = num.replace('[', '')
            num = num.replace(']', '')
            num = num.replace('\n', '')
            num = num.replace(',', '')
            num = int(num)
            nums.append(num)
        nums = np.array(nums, dtype='int')
        init_ids.append(nums)
print(len(training_num))
print(len(init_ids))

3570
51


In [1172]:
avg_sqlite_baseline=[0.8638879193162279, 0.8769568799248403, 0.866349147483102, 
                     0.9151616317551652, 0.9014299449681602, 0.9280529321935399, 0.899144850217508, 
                     0.9099761236367538, 0.9152248210642954, 0.9122596112857756, 0.8721451252580785, 
                     0.856024460470163, 0.9154533631642912, 0.8797699326925672, 0.8869232046826918, 
                     0.9360648680757941, 0.8677402351084885, 0.8342813381499408, 0.8712803643572351, 
                     0.8366687319478165, 0.9231964172593408, 0.8744842447005967, 0.9053573355231243, 
                     0.8672697988831533, 0.8292726199664983, 0.8749968985804153, 0.7783607755651424, 
                     0.7715601564972666, 0.7825533865868087, 0.9274488405917966, 0.7765864520122819, 
                     0.8791515930502445, 0.8406261685123564, 0.908374809899222, 0.8205414761334637, 
                     0.8998815011388299, 0.7527716371189755, 0.8111585662207265, 0.84417815397073, 
                     0.9047753760182994, 0.7367404795989195, 0.8300105942175335, 0.7730681258126023, 
                     0.7353831683166644, 0.8833862512127267, 0.8724714207658948, 0.8281741714363091, 
                     0.8656609292415972, 0.8548877930596626, 0.9014728113931414]

In [1173]:
training_num_set = np.array_split(training_num, 51)
training_num_set = np.array(training_num_set, dtype='int')

In [1175]:
index = training_num_set
r2_list=[]
r2_list_1=[]
l_num=[]
cut_num = 50
for commit_time in range(1, len(training_num_set)):
# for commit_time in range(24,27):  
    real_result = np.asarray(all_data[all_data['commit_num'] == commit_time]['time'])
    past_real_result = np.asarray(all_data[all_data['commit_num'] == commit_time-1]['time'])
    print('commit_time:' + str(commit_time))
    base_x_train = config_features[init_ids[commit_time]]
    base_y_train = past_real_result[init_ids[commit_time]]
    sub_selected_id = index[commit_time][35:70]
    print(len(sub_selected_id))
    # sub_selected_id = sub_selected_id[0:cut_num]
    # sub_selected_id = index[commit_time]
    for l in range(len(sub_selected_id)):
        extra_x_train = config_features[sub_selected_id[0:l]]
        extra_y_train = real_result[sub_selected_id[0:l]]
        X_train = np.concatenate((base_x_train,extra_x_train))
        y_train = np.concatenate((base_y_train,extra_y_train))
        X_test = config_features
        y_test = real_result
        model=XGBRegressor()
        model.fit(X_train, y_train)
        Y_predict = model.predict(X_test)
        # print('res_al: ' + str(res_al_all[commit_time]))
        # print('score: ' + str(model.score(X_test,y_test)))
        if model.score(X_test,y_test) > avg_sqlite_baseline[commit_time-1]:
            l_num.append(len(X_train)-len(init_ids[commit_time]))
            print('l: ' + str(l_num[-1]))
            print(model.score(X_test,y_test))
            print(avg_sqlite_baseline[commit_time-1])
            break
        #print(model.score(X_test,y_test))
        #r2_list.append(model.score(X_test,y_test))
        #print(res_al_all_sqlite[commit_time])
        #r2_list_1.append(res_al_all_sqlite[commit_time])
#if commit_time > 0 and len(sub_selected_id) > 0:
#    selected_id = sub_selected_id[0:cut_num]

commit_time:1
35
l: 0
0.864342459881123
0.8638879193162279
commit_time:2
35
l: 0
0.8798103168807194
0.8769568799248403
commit_time:3
35
l: 13
0.9810641875909567
0.866349147483102
commit_time:4
35
l: 12
0.9971258740152753
0.9151616317551652
commit_time:5
35
l: 6
0.985919599848409
0.9014299449681602
commit_time:6
35
commit_time:7
35
l: 6
0.9954346240037077
0.899144850217508
commit_time:8
35
l: 0
0.9949780147771264
0.9099761236367538
commit_time:9
35
l: 12
0.9953080658361529
0.9152248210642954
commit_time:10
35
l: 12
0.9972080239689183
0.9122596112857756
commit_time:11
35
l: 13
0.9939460301942891
0.8721451252580785
commit_time:12
35
commit_time:13
35
l: 5
0.9940976078504813
0.9154533631642912
commit_time:14
35
l: 14
0.882471873098983
0.8797699326925672
commit_time:15
35
commit_time:16
35
l: 11
0.9833287883597053
0.9360648680757941
commit_time:17
35
l: 0
0.8764796626239317
0.8677402351084885
commit_time:18
35
l: 0
0.8626656175044238
0.8342813381499408
commit_time:19
35
l: 0
0.9953525610306

In [1177]:
np.percentile(l_num,95)

16.39999999999999

In [1220]:
avg_lrzip_baseline=[0.799196542208493, 0.828288973980345, 0.8590695409455968, 0.8784447779839833, 
                    0.8694425998541974, 0.9367863891555035, 0.9057380845788973, 0.8378398638316159, 0.8988652851949184,
                    0.8649465637697857, 0.8298373402432532, 0.8887223713566238, 0.8483655713750963, 0.8280002126835764,
                    0.8719761060843686, 0.8757726929030454, 0.899894047182921, 0.8728102513739835, 0.8800158800740233,
                    0.8715108907899277, 0.8138674576652033, 0.8075126724888768, 0.8618638949204611, 0.9415236946973264,
                    0.8006644114800798, 0.9257540999475073, 0.8323265635668076, 0.8359221862985836, 0.8631684114178338,
                    0.8050763230415788, 0.7963768333671787, 0.8258421711987612, 0.8962609519649809, 0.8929948656890858,
                    0.8525502569350153, 0.8413337532184111, 0.8739378614381849, 0.8740014889555318, 0.891701732000339,
                    0.9200187109707496, 0.8909466436495089, 0.856679548679232, 0.8350823459411576, 0.8592386909881382,
                    0.8770350572358876, 0.8881577078066007, 0.9034584657520687, 0.8448523601606462, 0.8742618774384513,
                    0.9212717717710572, 0.8505719143571474, 0.8832005243912598, 0.779983929550607, 0.8660145124955007,
                    0.8634287588943821, 0.831866432882176, 0.8705433172690017, 0.820908564207273, 0.8707926727124687]

In [1242]:
file = open("lrzip.0", 'r')
lines = file.readlines()
i = 0
r2list = []
stdlist = []
for line in lines:
    # print(line)
    if line.__contains__('count'):
        i = i + 1
    if line.startswith('score'):
        # print(line[7:])
        # r2list.append([i, float(line[15:])])
        r2list.append([i, float(line[7:])])

In [1237]:
file = open("llvm.0", 'r')
lines = file.readlines()
i = 0
r2list = []
stdlist = []
for line in lines:
    # print(line)
    if line.__contains__('count'):
        i = i + 1
    if line.startswith('score'):
        # print(line[7:])
        # r2list.append([i, float(line[15:])])
        r2list.append([i, float(line[7:])])

In [1202]:
file = open("x264.0", 'r')
lines = file.readlines()
i = 0
r2list = []
stdlist = []
for line in lines:
    # print(line)
    if line.__contains__('count'):
        i = i + 1
    if line.startswith('score'):
        # print(line[7:])
        # r2list.append([i, float(line[15:])])
        r2list.append([i, float(line[7:])])
max_r2_list = []
for i in range(len(avg_x264_baseline)):
    test_line = []
    for j in range(0, len(r2list)):
        if r2list[j][0] == i:
            test_line.append(r2list[j][1])
    max_r2_list.append(max(test_line))
np.savetxt("x264_baseline.csv", avg_x264_baseline, delimiter=",")
np.savetxt("x264_con.csv", max_r2_list, delimiter=",")

In [1203]:
file = open("sqlite.0", 'r')
lines = file.readlines()
i = 0
r2list = []
stdlist = []
for line in lines:
    # print(line)
    if line.__contains__('count'):
        i = i + 1
    if line.startswith('score'):
        # print(line[7:])
        # r2list.append([i, float(line[15:])])
        r2list.append([i, float(line[7:])])
max_r2_list = []
for i in range(len(avg_sqlite_baseline)):
    test_line = []
    for j in range(0, len(r2list)):
        if r2list[j][0] == i:
            test_line.append(r2list[j][1])
    max_r2_list.append(max(test_line))